NOTE:
-----

Please run the below cells first before proceeding- you'll need them soon!

In [2]:
%load_ext sql
%sql sqlite:///dataset_1.db

/Users/tarabalakrishnan/.local/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/tarabalakrishnan/.local/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


u'Connected: None@dataset_1.db'

Activity 3-2
------------
Aggregation operators, GROUP BY

Exercise #1
-----------

Consider a set of tables that describe the up-and-coming bagel startup industry; for now let's just look at two tables here, `bagel`, which describes types of bagels made by the different bagel companies:
> * name STRING
> * price FLOAT
> * made_by STRING

And `purchase`:
> * bagel_name STRING
> * franchise STRING
> * date INT
> * quantity INT
> * purchaser_age INT

Where `purchase.bagel_name` references `bagel.name` and `purchase.franchise` references `bagel.made_by`:

In [9]:
%sql SELECT * FROM bagel

Done.


name,price,made_by
Plain with shmear,1.99,Bobs Bagels
Egg with shmear,2.39,Bobs Bagels
eBagel Drinkable Bagel,27.99,eBagel
eBagel Expansion Pack,1.99,eBagel
Plain with shmear,0.99,BAGEL CORP
Organic Flax-seed bagel chips,0.99,BAGEL CORP


In [8]:
%sql SELECT * FROM purchase

Done.


bagel_name,franchise,date,quantity,purchaser_age
Plain with shmear,Bobs Bagels,1,12,28
Egg with shmear,Bobs Bagels,2,6,47
Plain with shmear,BAGEL CORP,2,12,24
Plain with shmear,BAGEL CORP,3,1,17
eBagel Expansion Pack,eBagel,1,137,5
Plain with shmear,Bobs Bagels,4,24,None
Moonshine,BAGEL CORP,7,1000,37


Can you write a query to get the _total revenue_ for each bagel type **which had an average purchaser age over 18**?  Type your query below:

In [10]:
%%sql
SELECT b.name, SUM(p.quantity * b.price) AS Revenue
FROM bagel b, purchase p
WHERE b.name = p.bagel_name AND b.made_by = p.franchise
GROUP BY b.name
HAVING AVG(p.purchaser_age) > 18;

Done.


name,Revenue
Egg with shmear,14.34
Plain with shmear,84.51


Exercise #2
-----------

Here we'll use a simplified version of the `precipitation_full` table, which just has _daily_ rainfall _in CA only_, and has the following schema:

> * station_id
> * day
> * precipitation

In [5]:
%sql SELECT * FROM precipitation LIMIT 5;

Done.


station_id,day,precipitation
16102,1,10
16102,4,10
16102,24,30
21201,1,0
21201,20,10


We want to get station_ids which have average precipitations > 75.  Try doing this first as a nested query:

In [6]:
%%sql
SELECT DISTINCT p.station_id
FROM precipitation p 
WHERE (
    SELECT AVG(precipitation) 
    FROM precipitation 
    WHERE station_id = p.station_id) > 75;

Done.


station_id
88302
250002
335701
357302
488301


Now, try re-writing as a GROUP BY:

In [7]:
%%sql
SELECT station_id
FROM precipitation
GROUP BY station_id
HAVING AVG(precipitation) > 75;

Done.


station_id
88302
250002
335701
357302
488301


Now time it by using `%time` followed by single-line versions of your queries above (clunky, but will work) to see how they compare!

**Note:** Yes, currently the answers are filled in below for convenience... but you should still try getting them on your own above!

In [8]:
%time %sql SELECT DISTINCT p.station_id FROM precipitation p WHERE (SELECT AVG(precipitation) FROM precipitation WHERE station_id = p.station_id) > 75;

Done.
CPU times: user 41.9 ms, sys: 1.39 ms, total: 43.3 ms
Wall time: 42.5 ms


station_id
88302
250002
335701
357302
488301


In [9]:
%time %sql SELECT p.station_id FROM precipitation p GROUP BY p.station_id HAVING AVG(p.precipitation) > 75;

Done.
CPU times: user 3.06 ms, sys: 1.43 ms, total: 4.49 ms
Wall time: 3.11 ms


station_id
88302
250002
335701
357302
488301


**An ~ 10-20x difference in execution time!!**